# Advanced Feature Engineering for Credit Scoring - FIXED VERSION

This notebook creates powerful domain-specific features to improve model performance.

**Goal**: Improve ROC-AUC from 0.64 to 0.75+

In [8]:
# Cell 1: Import Libraries
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [9]:
# Cell 2: Load Raw Data
print("="*80)
print("LOADING RAW DATA")
print("="*80)

df = pd.read_parquet(
    r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\data\data-processing\flat_table\flat_credit_model_20251027_143321.parquet"
)

print(f"\nRaw data shape: {df.shape}")
print(f"Columns: {len(df.columns)}")
print(f"Default rate: {df['TARGET'].mean()*100:.2f}%")

print(f"\nClass distribution:")
print(df['TARGET'].value_counts())
print(df['TARGET'].value_counts(normalize=True))

LOADING RAW DATA

Raw data shape: (307511, 66)
Columns: 66
Default rate: 8.07%

Class distribution:
TARGET
False    282686
True      24825
Name: count, dtype: int64
TARGET
False    0.919271
True     0.080729
Name: proportion, dtype: float64

Raw data shape: (307511, 66)
Columns: 66
Default rate: 8.07%

Class distribution:
TARGET
False    282686
True      24825
Name: count, dtype: int64
TARGET
False    0.919271
True     0.080729
Name: proportion, dtype: float64


In [10]:
# Cell 3: Check Available Columns
print("="*80)
print("AVAILABLE COLUMNS IN DATASET")
print("="*80)

print(f"\nTotal columns: {len(df.columns)}")
print(f"\nAll columns:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

# Store column names for mapping
all_columns = df.columns.tolist()

AVAILABLE COLUMNS IN DATASET

Total columns: 66

All columns:
  1. SK_ID_CURR
  2. TARGET
  3. age_years
  4. employment_years
  5. annuity_income_ratio
  6. credit_income_ratio
  7. goods_income_ratio
  8. income_per_person
  9. has_job_flag
  10. raw_income_total
  11. raw_credit_amt
  12. raw_annuity_amt
  13. raw_goods_price
  14. raw_cnt_fam_members
  15. raw_days_employed
  16. app_missing_income_flag
  17. app_missing_credit_flag
  18. app_missing_annuity_flag
  19. app_missing_goods_flag
  20. total_credit_sum
  21. total_credit_debt
  22. total_utilization
  23. active_loans_count
  24. closed_loans_count
  25. max_overdue_ratio
  26. raw_bureau_records
  27. bur_raw_total_credit_sum
  28. bur_raw_total_credit_debt
  29. raw_total_overdue_amount
  30. raw_overdue_loans_count
  31. raw_has_overdue_flag
  32. cc_avg_utilization
  33. cc_max_utilization
  34. cc_payment_ratio
  35. cc_total_months
  36. cc_active_month_ratio
  37. cc_has_overdue_flag
  38. raw_cc_records
  39. cc

In [11]:
# Cell 4: Smart Column Mapper
print("="*80)
print("INTELLIGENT COLUMN MAPPING")
print("="*80)

def find_column(candidates, df_columns):
    """Find first matching column from candidate list"""
    for candidate in candidates:
        if candidate in df_columns:
            return candidate
    return None

# Map required columns with multiple alternatives
column_map = {
    'income': find_column(['income_total', 'raw_income_total', 'AMT_INCOME_TOTAL'], df.columns),
    'credit': find_column(['credit_amount', 'raw_credit_amt', 'AMT_CREDIT'], df.columns),
    'annuity': find_column(['annuity_amount', 'raw_annuity_amt', 'AMT_ANNUITY'], df.columns),
    'total_debt': find_column(['total_credit_debt', 'AMT_CREDIT_SUM_DEBT'], df.columns),
    'age_years': find_column(['age_years', 'DAYS_BIRTH'], df.columns),
    'employment': find_column(['employment_years', 'DAYS_EMPLOYED'], df.columns),
    'dpd_max': find_column(['dpd_max', 'pos_sk_dpd_max'], df.columns),
    'dpd_mean': find_column(['dpd_mean', 'pos_sk_dpd_mean'], df.columns),
    'dpd_std': find_column(['dpd_std', 'pos_sk_dpd_std'], df.columns),
}

print("\n📋 Column Mapping:")
for key, value in column_map.items():
    if value:
        print(f"  ✅ {key:15s} → {value}")
    else:
        print(f"  ❌ {key:15s} → NOT FOUND")

# Check which features we can create
can_create_debt = all([column_map['income'], column_map['credit'], column_map['total_debt']])
can_create_payment = all([column_map['dpd_max'], column_map['dpd_mean'], column_map['dpd_std']])

print(f"\n✅ Can create debt burden features: {can_create_debt}")
print(f"✅ Can create payment behavior features: {can_create_payment}")

INTELLIGENT COLUMN MAPPING

📋 Column Mapping:
  ✅ income          → raw_income_total
  ✅ credit          → raw_credit_amt
  ✅ annuity         → raw_annuity_amt
  ✅ total_debt      → total_credit_debt
  ✅ age_years       → age_years
  ✅ employment      → employment_years
  ✅ dpd_max         → dpd_max
  ✅ dpd_mean        → dpd_mean
  ❌ dpd_std         → NOT FOUND

✅ Can create debt burden features: True
✅ Can create payment behavior features: False


In [12]:
# Cell 5: Create Debt Burden Features
if can_create_debt:
    print("="*80)
    print("FEATURE ENGINEERING: DEBT BURDEN")
    print("="*80)
    
    income = df[column_map['income']].replace(0, np.nan)
    credit = df[column_map['credit']]
    total_debt = df[column_map['total_debt']]
    
    # 1. Total debt to income ratio
    df['total_debt_burden'] = total_debt / income
    
    # 2. New loan to income ratio
    df['new_loan_burden'] = credit / income
    
    # 3. Combined debt burden
    df['combined_debt_burden'] = (total_debt + credit) / income
    
    # 4. Income to loan ratio (inverse - higher is better)
    df['income_to_loan_ratio'] = income / credit.replace(0, np.nan)
    
    if column_map['annuity']:
        annuity = df[column_map['annuity']]
        # 5. Monthly payment burden
        df['monthly_payment_burden'] = ((annuity + total_debt / 12) / (income / 12))
    
    # Clip extreme values
    ratio_cols = ['total_debt_burden', 'new_loan_burden', 'combined_debt_burden', 
                  'income_to_loan_ratio']
    if column_map['annuity']:
        ratio_cols.append('monthly_payment_burden')
    
    df[ratio_cols] = df[ratio_cols].clip(lower=-10, upper=10)
    
    print(f"\n✅ Created {len(ratio_cols)} debt burden features")
    print(f"   Features: {', '.join(ratio_cols)}")
else:
    print("⚠️ Skipping debt burden features - required columns not found")

FEATURE ENGINEERING: DEBT BURDEN

✅ Created 5 debt burden features
   Features: total_debt_burden, new_loan_burden, combined_debt_burden, income_to_loan_ratio, monthly_payment_burden


In [13]:
# Cell 6: Create Payment Behavior Features
if can_create_payment:
    print("="*80)
    print("FEATURE ENGINEERING: PAYMENT BEHAVIOR")
    print("="*80)
    
    dpd_max = df[column_map['dpd_max']]
    dpd_mean = df[column_map['dpd_mean']]
    dpd_std = df[column_map['dpd_std']]
    
    # 1. Payment deterioration
    df['payment_deterioration'] = dpd_max - dpd_mean
    
    # 2. Payment consistency
    df['payment_consistency'] = 1 / (dpd_std + 1)
    
    # 3. Serious delinquency flag
    df['serious_delinquency'] = (dpd_max > 60).astype(int)
    
    # 4. Any delinquency flag
    df['any_delinquency'] = (dpd_max > 0).astype(int)
    
    # 5. DPD trend (max vs mean)
    df['dpd_trend'] = dpd_max / (dpd_mean + 1)
    
    payment_features = ['payment_deterioration', 'payment_consistency', 
                       'serious_delinquency', 'any_delinquency', 'dpd_trend']
    
    print(f"\n✅ Created {len(payment_features)} payment behavior features")
    print(f"   Features: {', '.join(payment_features)}")
else:
    print("⚠️ Skipping payment behavior features - required columns not found")

⚠️ Skipping payment behavior features - required columns not found


In [14]:
# Cell 7: Create Age & Employment Features
print("="*80)
print("FEATURE ENGINEERING: DEMOGRAPHICS")
print("="*80)

created_demo_features = []

if column_map['age_years']:
    age = df[column_map['age_years']]
    
    # 1. Age groups
    df['age_group'] = pd.cut(age, bins=[0, 25, 35, 45, 55, 100], 
                             labels=['<25', '25-35', '35-45', '45-55', '55+'])
    df['age_group'] = df['age_group'].cat.codes
    
    # 2. Prime age flag (30-50 years)
    df['prime_age'] = ((age >= 30) & (age <= 50)).astype(int)
    
    created_demo_features.extend(['age_group', 'prime_age'])

if column_map['employment']:
    employment = df[column_map['employment']]
    
    # 3. Employment stability
    df['employment_stable'] = (employment > 2).astype(int)  # >2 years
    
    created_demo_features.append('employment_stable')

if column_map['age_years'] and column_map['income']:
    # 4. Age-adjusted income
    df['age_adjusted_income'] = df[column_map['income']] / (age + 1)
    created_demo_features.append('age_adjusted_income')

if created_demo_features:
    print(f"\n✅ Created {len(created_demo_features)} demographic features")
    print(f"   Features: {', '.join(created_demo_features)}")
else:
    print("⚠️ No demographic features created - columns not available")

FEATURE ENGINEERING: DEMOGRAPHICS

✅ Created 4 demographic features
   Features: age_group, prime_age, employment_stable, age_adjusted_income

✅ Created 4 demographic features
   Features: age_group, prime_age, employment_stable, age_adjusted_income


In [15]:
# Cell 8: Create Interaction Features
print("="*80)
print("FEATURE ENGINEERING: INTERACTIONS")
print("="*80)

interaction_features = []

# Income x Age interaction
if column_map['income'] and column_map['age_years']:
    df['age_income_interaction'] = df[column_map['age_years']] * np.log1p(df[column_map['income']])
    interaction_features.append('age_income_interaction')

# DPD x Debt interaction
if can_create_payment and 'total_debt_burden' in df.columns:
    df['dpd_debt_risk'] = df[column_map['dpd_max']] * df['total_debt_burden']
    interaction_features.append('dpd_debt_risk')

# Credit amount x Employment
if column_map['credit'] and column_map['employment']:
    df['credit_employment_risk'] = df[column_map['credit']] / (df[column_map['employment']] + 1)
    interaction_features.append('credit_employment_risk')

if interaction_features:
    print(f"\n✅ Created {len(interaction_features)} interaction features")
    print(f"   Features: {', '.join(interaction_features)}")
else:
    print("⚠️ No interaction features created")

FEATURE ENGINEERING: INTERACTIONS

✅ Created 2 interaction features
   Features: age_income_interaction, credit_employment_risk

FEATURE ENGINEERING: INTERACTIONS

✅ Created 2 interaction features
   Features: age_income_interaction, credit_employment_risk


In [16]:
# Cell 9: Create Missing Value Indicators
print("="*80)
print("FEATURE ENGINEERING: MISSING VALUE FLAGS")
print("="*80)

# Key columns to track missing values
missing_track_cols = [col for col in [column_map['employment'], column_map['annuity'], 
                                       column_map['total_debt']] if col is not None]

missing_features = []
for col in missing_track_cols:
    if df[col].isnull().sum() > 0:
        feature_name = f'{col}_missing'
        df[feature_name] = df[col].isnull().astype(int)
        missing_features.append(feature_name)

if missing_features:
    print(f"\n✅ Created {len(missing_features)} missing value indicators")
    print(f"   Features: {', '.join(missing_features)}")
else:
    print("✅ No significant missing values to track")

FEATURE ENGINEERING: MISSING VALUE FLAGS

✅ Created 3 missing value indicators
   Features: employment_years_missing, raw_annuity_amt_missing, total_credit_debt_missing

✅ Created 3 missing value indicators
   Features: employment_years_missing, raw_annuity_amt_missing, total_credit_debt_missing


In [17]:
# Cell 10: Create Risk Score
print("="*80)
print("FEATURE ENGINEERING: COMPOSITE RISK SCORE")
print("="*80)

# Calculate composite risk score based on available features
risk_components = []

if 'total_debt_burden' in df.columns:
    risk_components.append(df['total_debt_burden'].fillna(0))

if 'any_delinquency' in df.columns:
    risk_components.append(df['any_delinquency'] * 2)  # Weight delinquency higher

if 'employment_stable' in df.columns:
    risk_components.append((1 - df['employment_stable']))  # Unstable = higher risk

if risk_components:
    df['composite_risk_score'] = sum(risk_components) / len(risk_components)
    print("\n✅ Created composite_risk_score")
    print(f"   Based on {len(risk_components)} components")
else:
    print("⚠️ Insufficient features for risk score")

FEATURE ENGINEERING: COMPOSITE RISK SCORE

✅ Created composite_risk_score
   Based on 2 components


In [18]:
# Cell 11: Summary of New Features
print("="*80)
print("FEATURE ENGINEERING SUMMARY")
print("="*80)

# Get original column count from v2 data
original_data = joblib.load(r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\output\models\processed_data_lgbm_v2.pkl")
original_features = original_data[0].shape[1]

# Get list of all new features
all_new_features = [col for col in df.columns if col not in all_columns and col != 'TARGET']

print(f"\n📊 Feature Statistics:")
print(f"   Original features: {original_features}")
print(f"   New features created: {len(all_new_features)}")
print(f"   Total features will be: {original_features + len(all_new_features)}")

print(f"\n✨ New Features Created:")
for i, feat in enumerate(all_new_features, 1):
    print(f"   {i:2d}. {feat}")

print(f"\n📈 Data Quality Check:")
print(f"   Total rows: {len(df):,}")
print(f"   Missing values in new features:")
for feat in all_new_features:
    missing_pct = df[feat].isnull().mean() * 100
    if missing_pct > 0:
        print(f"      {feat}: {missing_pct:.1f}%")

FEATURE ENGINEERING SUMMARY

📊 Feature Statistics:
   Original features: 62
   New features created: 15
   Total features will be: 77

✨ New Features Created:
    1. total_debt_burden
    2. new_loan_burden
    3. combined_debt_burden
    4. income_to_loan_ratio
    5. monthly_payment_burden
    6. age_group
    7. prime_age
    8. employment_stable
    9. age_adjusted_income
   10. age_income_interaction
   11. credit_employment_risk
   12. employment_years_missing
   13. raw_annuity_amt_missing
   14. total_credit_debt_missing
   15. composite_risk_score

📈 Data Quality Check:
   Total rows: 307,511
   Missing values in new features:
      total_debt_burden: 14.3%
      combined_debt_burden: 14.3%
      monthly_payment_burden: 14.3%
      credit_employment_risk: 18.0%

📊 Feature Statistics:
   Original features: 62
   New features created: 15
   Total features will be: 77

✨ New Features Created:
    1. total_debt_burden
    2. new_loan_burden
    3. combined_debt_burden
    4. incom

In [19]:
# Cell 12: Prepare Data for Training
print("="*80)
print("PREPARING ENHANCED DATASET")
print("="*80)

# Separate features and target
X = df.drop(columns=['TARGET', 'SK_ID_CURR'] if 'SK_ID_CURR' in df.columns else ['TARGET'])
y = df['TARGET']

# Handle any remaining missing values
print(f"\nHandling missing values...")
X = X.fillna(X.median())

# Handle any infinite values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

print(f"\n✅ Data preparation complete")
print(f"   Feature matrix shape: {X.shape}")
print(f"   Target shape: {y.shape}")
print(f"   Class distribution: {Counter(y)}")

PREPARING ENHANCED DATASET

Handling missing values...

✅ Data preparation complete
   Feature matrix shape: (307511, 79)
   Target shape: (307511,)
   Class distribution: Counter({False: 282686, True: 24825})

✅ Data preparation complete
   Feature matrix shape: (307511, 79)
   Target shape: (307511,)
   Class distribution: Counter({False: 282686, True: 24825})


In [20]:
# Cell 13: Train-Test Split
print("="*80)
print("CREATING TRAIN-TEST SPLIT")
print("="*80)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"\n✅ Data split complete:")
print(f"   Train: {X_train.shape}")
print(f"   Test:  {X_test.shape}")
print(f"\n   Train class distribution:")
print(f"      Class 0: {Counter(y_train)[0]:,} ({Counter(y_train)[0]/len(y_train)*100:.1f}%)")
print(f"      Class 1: {Counter(y_train)[1]:,} ({Counter(y_train)[1]/len(y_train)*100:.1f}%)")

CREATING TRAIN-TEST SPLIT

✅ Data split complete:
   Train: (246008, 79)
   Test:  (61503, 79)

   Train class distribution:
      Class 0: 226,148 (91.9%)
      Class 1: 19,860 (8.1%)

✅ Data split complete:
   Train: (246008, 79)
   Test:  (61503, 79)

   Train class distribution:
      Class 0: 226,148 (91.9%)
      Class 1: 19,860 (8.1%)


In [21]:
# Cell 14: Quick Baseline Test
print("="*80)
print("QUICK BASELINE TEST WITH NEW FEATURES")
print("="*80)

# Train quick model to verify features work
lgb_test = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=31,
    random_state=42,
    n_jobs=-1
)

print("\n🔄 Training quick baseline...")
lgb_test.fit(X_train, y_train)

# Evaluate
y_prob_test = lgb_test.predict_proba(X_test)[:, 1]
baseline_auc = roc_auc_score(y_test, y_prob_test)

# Load old AUC for comparison
old_metadata = joblib.load(r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\output\models\improved_model_metadata.pkl")
old_auc = old_metadata['roc_auc']

improvement = ((baseline_auc - old_auc) / old_auc) * 100

print(f"\n📊 Quick Test Results:")
print(f"   Old model AUC: {old_auc:.4f}")
print(f"   New features AUC: {baseline_auc:.4f}")
print(f"   Improvement: {improvement:+.2f}%")

if baseline_auc > old_auc:
    print(f"\n✅ SUCCESS! New features improved performance!")
elif baseline_auc > 0.70:
    print(f"\n✅ GOOD! AUC > 0.70 with minimal training")
else:
    print(f"\n⚠️ WARNING: Performance not improved yet")
    print(f"   This may improve with full hyperparameter tuning")

QUICK BASELINE TEST WITH NEW FEATURES

🔄 Training quick baseline...
[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.088072 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12551
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 71
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432482
[LightGBM] [Info] Start training from score -2.432482
[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.088072 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12551
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 71
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2

In [22]:
# Cell 15: Feature Importance Analysis
print("="*80)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*80)

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgb_test.feature_importances_
}).sort_values('importance', ascending=False)

# Highlight new features
feature_importance['is_new'] = feature_importance['feature'].isin(all_new_features)

print(f"\n📊 Top 20 Most Important Features:")
print("="*80)
for idx, row in feature_importance.head(20).iterrows():
    marker = "🆕" if row['is_new'] else "  "
    print(f"{marker} {row['feature']:35s} {row['importance']:8.1f}")

# Count new features in top rankings
new_in_top20 = feature_importance.head(20)['is_new'].sum()
new_in_top50 = feature_importance.head(50)['is_new'].sum()

print(f"\n📈 New Feature Impact:")
print(f"   New features in top 20: {new_in_top20}/{len(all_new_features)}")
print(f"   New features in top 50: {new_in_top50}/{len(all_new_features)}")
print(f"   Total new features: {len(all_new_features)}")

FEATURE IMPORTANCE ANALYSIS

📊 Top 20 Most Important Features:
   active_loans_count                     143.0
   raw_goods_price                        134.0
   raw_annuity_amt                        133.0
   total_utilization                      131.0
   raw_credit_amt                         127.0
   total_credit_sum                       117.0
   annuity_income_ratio                    97.0
🆕 credit_employment_risk                  89.0
   ins_raw_total_instalment                88.0
   on_time_ratio                           85.0
🆕 age_adjusted_income                     83.0
   ins_raw_on_time_count                   71.0
🆕 age_income_interaction                  69.0
   cc_max_utilization                      66.0
   ins_raw_total_payment                   65.0
   age_years                               64.0
   employment_years                        62.0
   cc_payment_ratio                        62.0
   ins_payment_variance                    58.0
   ins_early_ratio          

In [23]:
# Cell 16: Save Enhanced Data
print("="*80)
print("SAVING ENHANCED DATA")
print("="*80)

# Save processed data
output_path = r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\output\models\processed_data_lgbm_v3.pkl"
joblib.dump((X_train, X_test, y_train, y_test), output_path)

# Save metadata
metadata = {
    'creation_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'original_features': original_features,
    'new_features_created': len(all_new_features),
    'total_features': X_train.shape[1],
    'new_feature_list': all_new_features,
    'baseline_auc': baseline_auc,
    'old_auc': old_auc,
    'improvement_vs_old': improvement,
    'train_shape': X_train.shape,
    'test_shape': X_test.shape,
    'class_distribution': {
        'train': Counter(y_train),
        'test': Counter(y_test)
    }
}

metadata_path = r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\output\models\enhanced_data_metadata.pkl"
joblib.dump(metadata, metadata_path)

print(f"\n✅ Enhanced data saved successfully!")
print(f"   Data: processed_data_lgbm_v3.pkl")
print(f"   Metadata: enhanced_data_metadata.pkl")
print(f"\n📊 Summary:")
print(f"   Total features: {X_train.shape[1]}")
print(f"   New features: {len(all_new_features)}")
print(f"   Baseline AUC: {baseline_auc:.4f}")
print(f"   Improvement: {improvement:+.2f}%")
print(f"\n🎯 Next Step: Re-run 04b_baseline_model_LightGBM_improved.ipynb")

SAVING ENHANCED DATA

✅ Enhanced data saved successfully!
   Data: processed_data_lgbm_v3.pkl
   Metadata: enhanced_data_metadata.pkl

📊 Summary:
   Total features: 79
   New features: 15
   Baseline AUC: 0.7078
   Improvement: +9.47%

🎯 Next Step: Re-run 04b_baseline_model_LightGBM_improved.ipynb

✅ Enhanced data saved successfully!
   Data: processed_data_lgbm_v3.pkl
   Metadata: enhanced_data_metadata.pkl

📊 Summary:
   Total features: 79
   New features: 15
   Baseline AUC: 0.7078
   Improvement: +9.47%

🎯 Next Step: Re-run 04b_baseline_model_LightGBM_improved.ipynb
